This is a page for exploring the STAC collections available for a given site (sub-catalog)

In [ ]:
from operator import attrgetter

import panel as pn
import param

import utils.cards
import utils.location
import utils.template
import utils.utils

pn.extension(loading_indicator=True, defer_load=True)

In [ ]:
class Page(utils.location.UrlQueryParams):
    @param.depends("site", watch=False)
    def get_collection_cards(self) -> list:
        if self.site is None:
            return ["No collections found"]
        collections = utils.utils.get_collections(self.site)
        collections = sorted(collections, key=attrgetter("title", "id"))
        return [utils.cards.collection(self.site, c, with_links=True, collapsed=True) for c in collections]

    @param.depends("site", watch=False)
    def main(self) -> pn.Column:
        """content to render in the main area of a template"""
        elements = []

        if self.site is None:
            elements.append(pn.pane.Alert("Site not found", alert_type="danger"))
        else:
            site_name = self.site.title or self.site.id
            elements.append(
                pn.pane.Markdown(
                    f"""
                    # Collections

                    On this page you can discover the data collections available for the site {site_name}.
                    Each collection contains a specific type of data related to peat health,
                    such as multi-year timeseries of biogeophysical, hydrological, and surface motion variables.

                    Check out one of the collections below to explore the data.
                    """
                )
            )
            elements.append(pn.pane.Markdown("## Site"))
            elements.append(utils.cards.site(self.site, collapsed=True))
            elements.append(pn.pane.Markdown("## Data collections"))
            elements.extend(self.get_collection_cards())  # type: ignore

        return pn.Column(
            *elements,
            align=("start", "center"),
            min_width=400,
            max_width=1200,
            sizing_mode="stretch_width",
        )


In [ ]:
page = Page()

In [ ]:
# In a notebook environment pn.state.location is not initialized until the first plot has been displayed

pn.state.location.sync(page, {"site_id": "site-id"})  # type: ignore

In [ ]:
template = utils.template.get_template(main=page.main)
template.servable();